In [1]:
import cv2
import mediapipe as mp
import datetime as dt
import time


mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For static images:
IMAGE_FILES = []
with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5) as face_detection:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw face detections of each face.
    if not results.detections:
      continue
    annotated_image = image.copy()
    for detection in results.detections:
      print('Nose tip:')
      print(mp_face_detection.get_key_point(
          detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
      mp_drawing.draw_detection(annotated_image, detection)
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

# For webcam input:
cap = cv2.VideoCapture(0)
tracking_points=[]
inicio = time.time()
image_trayectoria = None
print(inicio)
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
  while cap.isOpened():
    if float(time.time()-inicio)>30.0:
        break
    success, image = cap.read()
    image_trayectoria = image
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image)
    
    
    if results.detections:
        detection = results.detections[0]
        xmin = detection.location_data.relative_bounding_box.xmin
        ymin = detection.location_data.relative_bounding_box.ymin
        time_ = dt.datetime.now().strftime('%H:%M:%S')
        
        tracking_point = {
            'time':time_,
            'xmin':xmin,
            'ymin':ymin
        }
        
    tracking_points.append(tracking_point)

        
    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.detections:
      for detection in results.detections:
        mp_drawing.draw_detection(image, detection)

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
  cap.release()

  print(len(tracking_points))

  for i in tracking_points:
      x_center = i['xmin']
      y_center = i['ymin']
      image = cv2.circle(image,(x_center,y_center),2,(0,0,255))
    
  cv2.imshow('Trayectoria',cv2.flip(image, 1))

1648298870.5285726
883


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type


In [ ]:
import pandas as pd
df = pd.DataFrame(tracking_points)
df = df.set_index("time")
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize = (9, 5), sharey = True)
ax.scatter(x=df.xmin,y=df.ymin)

sns.despine()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (12, 5), sharey = False)
ax1 = df.ymin.plot()
ax2 = df.xmin.plot()

plt.legend()
sns.despine()